# Building a Linear Regression Model to predict Housing Prices

In [1]:
import numpy as np
import pandas as pd

In [2]:
houses = pd.read_csv('kc_house_data.csv')

In [3]:
houses.head(3)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062


In [4]:
houses.columns

Index(['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'sqft_living15', 'sqft_lot15'],
      dtype='object')

## Section 1

Remove these columns:
<ul>
<li><b>id</b>: Numbers have no other meaning outside of unit identification
<li><b>view</b>: Just because someone viewed house, doesn't mean he would like it more
<li><b>zipcode</b>: big / small does not mean better; just an ID number
<li><b>lat</b>: bigger or smaller doesn't mean better
<li><b>long</b>: bigger or smaller doesn't mean better
<li><b>sqft_living15</b>: useless for 2014 houses 
<li><b>sqft_lot15</b>: useless for 2014 houses
<li><b>yr_renovated</b>: many are blank if never renovated
<li><b>sqft_basement</b>: not as important as other parts of house; could be blank if no basement
</ul>

In [5]:
df1 = houses.copy()

In [6]:
cols = df1.columns.get_values().tolist()
UselessCols = ['id','view','zipcode','lat','long','sqft_living15','sqft_lot15', 'yr_renovated','sqft_basement']

goodCols = set(cols).difference(UselessCols)      # remove useless columns
goodCols = list(goodCols)                         # convert to list

In [7]:
df1.drop(UselessCols,axis=1,inplace=True)

Split date into months & years

In [8]:
#df1['date_Year'] = DF['date'].apply(lambda x: x[0:4])
#df1['date_Month'] = DF['date'].apply(lambda x: x[4:6])
df1.drop(['date'],axis=1,inplace=True)

Top 10 predictors (excluding price):

In [9]:
df1.columns

Index(['price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
       'waterfront', 'condition', 'grade', 'sqft_above', 'yr_built'],
      dtype='object')

In [10]:
# Export for RStudio
pd.DataFrame.to_csv(df1,'mcchowHW2_PT1_df1.csv',index=False)

Use forward selection to build model with 10 predictors

In [11]:
import statsmodels.formula.api as smf

# function definition from <https://planspace.org/20150423-forward_selection_with_statsmodels/>

def forward_selected(data, response):
    """Linear model designed by forward selection.

    Parameters:
    -----------
    data : pandas DataFrame with all possible predictors and response

    response: string, name of response column in data

    Returns:
    --------
    model: an "optimal" fitted statsmodels linear model
           with an intercept
           selected by forward selection
           evaluated by adjusted R-squared
    """
    remaining = set(data.columns)
    remaining.remove(response)
    selected = []
    current_score, best_new_score = 0.0, 0.0
    while remaining and current_score == best_new_score:
        scores_with_candidates = []
        for candidate in remaining:
            formula = "{} ~ {} + 1".format(response,
                                           ' + '.join(selected + [candidate]))
            score = smf.ols(formula, data).fit().rsquared_adj
            scores_with_candidates.append((score, candidate))
        scores_with_candidates.sort()
        best_new_score, best_candidate = scores_with_candidates.pop()
        if current_score < best_new_score:
            remaining.remove(best_candidate)
            selected.append(best_candidate)
            current_score = best_new_score
    formula = "{} ~ {} + 1".format(response,
                                   ' + '.join(selected))
    model = smf.ols(formula, data).fit()
    return model

In [12]:
model = forward_selected(df1,'price')

In [13]:
print(model.model.formula)

price ~ sqft_living + grade + yr_built + waterfront + bedrooms + bathrooms + sqft_lot + condition + floors + sqft_above + 1


In [14]:
print(model.rsquared_adj)

0.645545065855


This R2 is good, because in a market with so many variables to price, being able to explain over 50% of variance is powerful.

## Section 2

In [15]:
df2 = houses.copy()

Turn zipcode into category.  
This will make zipcode a useful predictor because houses that share zipcodes tend to share similar prices.  
Location category is a big factor (ie: near a big city?)

In [16]:
df2['zipcode_cat'] = df2.zipcode.astype('category')

New set of top 10 predictors:

In [17]:
UselessCols2 = ['date','id','view','yr_built','lat','long','sqft_living15','sqft_lot15', 'yr_renovated','sqft_basement', 'zipcode']
df2.drop(UselessCols2, axis=1, inplace=True)
df2.columns

Index(['price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
       'waterfront', 'condition', 'grade', 'sqft_above', 'zipcode_cat'],
      dtype='object')

In [21]:
# Export for RStudio
pd.DataFrame.to_csv(df2,'mcchowHW2_PT1_df2.csv',index=False)

Building model:

In [18]:
model2 = forward_selected(df2,'price')

In [19]:
print(model2.model.formula)

price ~ sqft_living + zipcode_cat + waterfront + grade + bedrooms + condition + floors + sqft_above + sqft_lot + bathrooms + 1


In [20]:
print(model2.rsquared_adj)

0.794929695913


R2 has significantly improved from simply swapping out 'yr_built' for 'zipcode_cat'

### heteroscedasticity test:

In [22]:
hed = pd.DataFrame(data=model2.fittedvalues, columns={'fitted'})

In [23]:
hed['nobs'] = model2.nobs  # nobs = num rows

In [24]:
hed.head(2)

,fitted,nobs
0,194477.230697,21613.0
1,590854.429761,21613.0


In [25]:
compar = []

for i in range(0, len(hed)):
    compar.append([hed.fitted[i],hed.nobs[i]])

In [26]:
import statsmodels.stats.diagnostic as smd

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [27]:
smd.het_breuschpagan(model2.resid, compar)

(1959.4514095337001, 0.0, 2154.6085795403974, 0.0)